In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time

%matplotlib widget

In [2]:
from bluesky_tutorial_utils import setup_data_saving
from bluesky import RunEngine
from bluesky.plans import count
from utils import simple_integration

RE = RunEngine()
catalog = setup_data_saving(RE)

In [3]:
from simulated_hardware import detector, load_sample, unload_sample, light, history_reset, sim_sleep
from simulated_hardware import _history

--- Part C: Optional "High-Score" mode

1.) Say you want to measure sample multiple times, add up results to improve statistics

2.) Demonstrate how to do this (and that it improves things for weakly scattering samples)

3.) Explain that in real life, can't just measure forever, need to prioritize and decide how to use your beamtime

4.) Game is to get the 'best score' by arranging data collection strategy over a limited period of time, compare to ideal data.

5.) High-score list???


Make perfect data for comparison

In [23]:
_history['perfect_data'] = True

perfect_pair = RE(dark_light_subtract(1))    

this_light = catalog[perfect_pair[1]].primary.read().detector_image[0]

perfect_int = simple_integration(this_light,num_bins=301)

_history['perfect_data'] = False


### Averaging Data

In [5]:
def dark_light_subtract(sample_num, num_lights = 1):
    #close shutter if not already closed
    yield from light(False)
        
    #move to desired sample
    yield from load_sample(sample_num)
    
    #take dark image
    yield from count([detector])
    
    #open shutter
    yield from light(True)
    
    #take light image
    for i in range(num_lights):
        yield from count([detector])
    
    #close shutter to be nice to detector
    yield from light(False)
    

def process_data(pair, num_lights = 1):
    #assuming pair is tuple
    my_dark = catalog[pair[0]].primary.read().detector_image[0]
    
    dark_subbed_list = []
    
    for i in range(1,num_lights+1):
        this_light = catalog[pair[i]].primary.read().detector_image[0]
        dark_subbed_list.append(this_light - my_dark)
    
    if num_lights == 1:
        return simple_integration(dark_subbed_list[0],num_bins=301)
    
    else: # more than one
        int_list = []
        for j in range(len(dark_subbed_list)):
            int_list.append(simple_integration(dark_subbed_list[j],num_bins=301))
    
        return np.array(int_list).T

In [26]:
_history['noise'] = 1000

In [27]:
_history['action_time'] = 20.0 # 1.0
_history['panel_wl'] = 3000 # 8000

In [28]:
uids1 = RE(dark_light_subtract(1, num_lights=15))

In [29]:
uids1

('5805173b-2ef5-4c99-8e28-09fc2ffdd627',
 '393d4388-f373-47f0-a99b-e306b9efebc7',
 '69f136a9-8092-43f4-a0bd-e0ba87fe47fa',
 'bea35d2e-2275-4134-a0be-65fb349c6bae',
 '96919b5c-ef6e-4de2-a15b-a9221bd679ef',
 '9ae3d626-cc43-46ef-adb3-6a8221579f76',
 '9d86933a-5ea2-4fb3-8e8f-3afbdc3c4fc6',
 '7d529cfc-b6ed-4004-bf1f-2b62768af1d9',
 '5ca7ee2b-2413-4bbe-abba-2e3768e506fa',
 '9657f23d-acdb-482e-8fed-a12be545f249',
 '82593861-840c-4233-a766-1abfb2ef7bc7',
 '33467d20-41f2-4865-b537-02efdf704375',
 'c2c0401e-7fb2-45da-bb56-beda6c642ef5',
 '9f04d306-87fe-46ca-9cb2-a859a9fa670e',
 '5c11b205-19a3-4652-9de0-599054665ba8',
 '69bc36ff-1137-4eb4-b6e7-90f0d15c86ed')

In [30]:
plt.close('all')

In [31]:
plt.figure()
intensity_list = process_data(uids1, num_lights=len(uids1)-1)
plt.plot(intensity_list);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.figure()
#plt.plot(intensity_list.mean(axis=1))
plt.plot(perfect_int)
plt.plot(intensity_list.T[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
history_reset()

RE(light(True))

RE(load_sample(1))

RE(load_sample(2))

RE(load_sample(3))

### Introductory Time Travel

Goal is to develop a plan that measures samples as efficiently as possible, and you get a score based on goodness of fit by averaging data.